In [24]:
class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid
        self.variables = [(i, j) for i in range(9) for j in range(9)]
        self.domains = {
            var: set(range(1, 10)) if grid[var[0]][var[1]] == 0 else {grid[var[0]][var[1]]}
            for var in self.variables
        }
        self.constraints = self.generate_constraints()

    def generate_constraints(self):

        constraints = {}
        for i in range(9):
            for j in range(9):
                if (i, j) not in constraints:
                    constraints[(i, j)] = set()
                # Row constraints
                for col in range(9):
                    if col != j:
                        constraints[(i, j)].add((i, col))
                # Column constraints
                for row in range(9):
                    if row != i:
                        constraints[(i, j)].add((row, j))
                # Subgrid constraints
                row_start = (i // 3) * 3
                col_start = (j // 3) * 3
                for r in range(row_start, row_start + 3):
                    for c in range(col_start, col_start + 3):
                        if (r, c) != (i, j):
                            constraints[(i, j)].add((r, c))
        return constraints

    def is_valid(self, var, value):

        i, j = var
        for neighbor in self.constraints[var]:
            if self.grid[neighbor[0]][neighbor[1]] == value:
                return False
        return True

    def forward_checking(self, var, value):
        affected = []
        for neighbor in self.constraints[var]:
            r, c = neighbor
            if self.grid[r][c] == 0 and value in self.domains[neighbor]:
                self.domains[neighbor].remove(value)
                affected.append((neighbor, value))
                if len(self.domains[neighbor]) == 0:
                    self.restore_domains(affected, value)
                    return None
        return affected

    def restore_domains(self, affected, value):
        for neighbor, val in affected:
            self.domains[neighbor].add(val)

    def backtrack(self):
        unassigned = [(var, self.domains[var]) for var in self.variables if self.grid[var[0]][var[1]] == 0]
        if not unassigned:
            return True

        var = min(unassigned, key=lambda x: len(x[1]))[0]
        for value in list(self.domains[var]):
            if self.is_valid(var, value):
                self.grid[var[0]][var[1]] = value
                saved_domain = self.domains[var].copy()
                self.domains[var] = {value}
                affected = self.forward_checking(var, value)
                if affected is not None:
                    if self.backtrack():
                        return True
                self.grid[var[0]][var[1]] = 0
                self.domains[var] = saved_domain
                if affected is not None:
                    self.restore_domains(affected, value)
        return False

    def solve(self):
        if self.backtrack():
            return self.grid
        else:
            return None

In [23]:

# 9x9 Sudoku Grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

# Solve the Sudoku
solver = SudokuSolver(sudoku_grid)
solution = solver.solve()

# Print the solution
if solution:
    print("Solved Sudoku:")
    for row in solution:
        print(row)
else:
    print("No solution exists.")


Solved Sudoku:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
